# Linear regression implementations

In [ ]:
# set up working catalog
import sys
from pathlib import Path
project_path = str(Path().cwd().parent.resolve())
if project_path not in sys.path:
    sys.path.append(project_path)

# imports
from common.utils import *
from common.custom_linear_regression import *

import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [13]:
data = get_data(only_numeric=True)

X = data.drop("Admission grade", axis=1)
X = StandardScaler().fit_transform(X)       # normalize
y = data["Admission grade"]

X_train, X_val, X_test, y_train, y_val, y_test = split(X, y)

data.head()

,Previous qualification (grade),Admission grade,Age at enrollment,Curricular units 1st sem (credited),Curricular units 1st sem (enrolled),Curricular units 1st sem (evaluations),Curricular units 1st sem (approved),Curricular units 1st sem (grade),Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,122.0,127.3,20,0,0,0,0,0.000000,0,0,0,0,0,0.000000,0,10.8,1.4,1.74
1,160.0,142.5,19,0,6,6,6,14.000000,0,0,6,6,6,13.666667,0,13.9,-0.3,0.79
2,122.0,124.8,19,0,6,0,0,0.000000,0,0,6,0,0,0.000000,0,10.8,1.4,1.74
3,122.0,119.6,20,0,6,8,6,13.428571,0,0,6,10,5,12.400000,0,9.4,-0.8,-3.12
4,100.0,141.5,45,0,6,9,5,12.333333,0,0,6,6,6,13.000000,0,13.9,-0.3,0.79


In [14]:
def get_result(model, **kwargs):
    model.fit(X_train, y_train, **kwargs)
    y_pred_train = model.predict(X_train)
    y_pred_val = model.predict(X_val)
    y_pred_test = model.predict(X_test)
    
    return {
        "train": (mean_squared_error(y_train, y_pred_train), r2_score(y_train, y_pred_train)),
        "val": (mean_squared_error(y_val, y_pred_val), r2_score(y_val, y_pred_val)),
        "test": (mean_squared_error(y_test, y_pred_test), r2_score(y_test, y_pred_test)),
    }
    
def print_result(result):
    print(
        f"\tTrain:\t\tMSE: {result['train'][0]:.4f}",
        f"\tR2: {result['train'][1]:.4f}"
    )
    print(
        f"\tValidation:\tMSE: {result['val'][0]:.4f}",
        f"\tR2: {result['val'][1]:.4f}"
    )
    print(
        f"\tTrain:\t\tMSE: {result['test'][0]:.4f}",
        f"\tR2: {result['test'][1]:.4f}"
    )
    

In [15]:
print("===== FINDING BEST PARAMS =====")
best_params = find_gradient_descent_best_params(X_train, y_train, X_val, y_val, should_log=True)
print(f"Best params: {best_params}")

print("\n===== GRADIENT DESCENT LINEAR REGRESSION =====")
gradient_result = get_result(LinearRegressionGradientDescent(), **best_params)
print_result(gradient_result)

===== FINDING BEST PARAMS =====
	epochs =  2500
	epochs =  5000
	epochs =  7500
	epochs =  10000
Best params: {'epochs': 2500, 'batch_size': 64, 'learning_rate': 0.005}

===== GRADIENT DESCENT LINEAR REGRESSION =====
	Train:		MSE: 131.0669 	R2: 0.3593
	Validation:	MSE: 123.4209 	R2: 0.4475
	Train:		MSE: 152.6976 	R2: 0.3062


In [16]:
print("===== CLOSED FORM SOLUTION LINEAR REGRESSION =====")
closed_result = get_result(LinearRegressionClosedForm())
print_result(closed_result)

===== CLOSED FORM SOLUTION LINEAR REGRESSION =====
	Train:		MSE: 130.9588 	R2: 0.3598
	Validation:	MSE: 124.2138 	R2: 0.4440
	Train:		MSE: 152.8460 	R2: 0.3056


In [17]:
print("\n===== SKLEARN LINEAR REGRESSION =====")
sklearn_result = get_result(LinearRegression())
print_result(sklearn_result)


===== SKLEARN LINEAR REGRESSION =====
	Train:		MSE: 130.9588 	R2: 0.3598
	Validation:	MSE: 124.2138 	R2: 0.4440
	Train:		MSE: 152.8460 	R2: 0.3056


In [18]:
df = pd.DataFrame({
    "Train MSE": [gradient_result['train'][0], closed_result['train'][0], sklearn_result['train'][0]],
    "Train R²": [gradient_result['train'][1], closed_result['train'][1], sklearn_result['train'][1]],
    "Val MSE": [gradient_result['val'][0], closed_result['val'][0], sklearn_result['val'][0]],
    "Val R²": [gradient_result['val'][1], closed_result['val'][1], sklearn_result['val'][1]],
    "Test MSE": [gradient_result['test'][0], closed_result['test'][0], sklearn_result['test'][0]],
    "Test R²": [gradient_result['test'][1], closed_result['test'][1], sklearn_result['test'][1]],
}, index=["Gradient descent", "Closed form solution", "Sklearn"])

df

,Train MSE,Train R²,Val MSE,Val R²,Test MSE,Test R²
Gradient descent,131.066901,0.359313,123.420913,0.447531,152.697588,0.306231
Closed form solution,130.958762,0.359841,124.213812,0.443981,152.845965,0.305557
Sklearn,130.958762,0.359841,124.213812,0.443981,152.845965,0.305557


# Conclusions
1) The results are basically the same for sklearn and closed form solution implementation
2) Gradient descent gives nearly the same results